# QENS data reduction - DMSC summer school

In [ ]:
import numpy as np
import plopp as pp
import scipp as sc
import scippneutron as scn

import utils

In [ ]:
%matplotlib widget

## Load raw data

In [ ]:
# Optional: download the data
#!wget --no-verbose https://public.esss.dk/groups/scipp/dmsc-summer-school/qens/qens_known_quasi_elastic_1_pulse.h5

In [ ]:
fname = 'qens_known_quasi_elastic_1_pulse.h5'
events = utils.load_qens(fname)

In [ ]:
events

## Inspect raw data

In [ ]:
source_settings = {
    'center': events.coords['source_position'],
    'color': '#FFC133',
    'size': sc.vector(value=[10, 20, 10], unit=sc.units.m),
    'type': 'cylinder',
}
sample_settings = {
    'center': events.meta['sample_position'],
    'color': '#000000',
    'wireframe': True,
    'size': sc.vector(value=[0.01, 0.01, 0.01], unit=sc.units.m),
    'type': 'box',
}
analyzer_settings = {
    'center': events.coords['analyzer_position'],
    'color': '#AA0000',
    'size': sc.vector([0.1, 0.1, 0.1], unit='m'),
    'type': 'box',
}
scn.instrument_view(
    events,
    components={'analyzer': analyzer_settings, 'source': source_settings, 'sample': sample_settings})

In [ ]:
events.hist(tof=100, y=100).plot()

## Mask bad region

In [ ]:
y = events.coords['y']
binned_for_mask = events.bin(y=sc.array(dims=['y'],
                           values=[
                               y.min().value,
                               0.2615,
                               0.27,
                               np.nextafter(y.max().value, np.inf)
                           ],
                           unit=y.unit))
mask = sc.array(dims=['y'], values=[False, True, False])
binned_for_mask.masks['bad_timing'] = mask
binned_for_mask

In [ ]:
binned_for_mask.hist().plot()

In [ ]:
binned_for_mask.hist(tof=100, y=100).plot()

## Transform to energ transfer

In [ ]:
def backscattered_l2(position, sample_position, analyzer_position):
    """
    Compute the length of the secondary flight path for backscattering off an analyzer.
    """
    # TODO Subtraction from Mad's script ?!
    return sc.norm(position-analyzer_position) + sc.norm(analyzer_position-sample_position) - sc.scalar(0.03, unit='m')


def wavelength_from_analyzer(analyzer_dspacing, analyzer_angle):
    """
    Compute the neutron wavelength after scattering from the analyzer's d-spacing.

    Assuming Bragg scattering in the analyzer, the wavelength is
        wavelength = 2 * d * sin(theta)

    Where
        d is the analyzer's d-spacing,
        theta is the scattering angle or equivalently, the tilt of the analyzer
              w.r.t. to the sample-analyzer axis.
    """
    # 2*theta is the angle between transmitted and scattered beam.
    # So because of backscattering, we need to subtract the analyzer angle from 2*pi.
    return 2 * analyzer_dspacing * sc.sin(sc.scalar(np.pi/2, unit='rad') - analyzer_angle.to(unit='rad'))


def final_energy(final_wavelength):
    """
    Compute the neutron energy after scattering.

    Uses
        final_energy = mn / 2 * final_speed**2
        final_speed = 2 * pi * hbar / mn / final_wavelength

    Where
        mn is the neutron mass,
        final_wavelength is the wavelength after scattering,
        final_speed is the speed after scattering.
    """
    return sc.to_unit(sc.constants.h**2/ 2 / sc.constants.neutron_mass / (final_wavelength ** 2),
                      'meV')

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.tof import energy_transfer_indirect_from_tof

graph = {
    **beamline(scatter=True),
    'L2': backscattered_l2,
    'final_wavelength': wavelength_from_analyzer,
    'final_energy': final_energy,
    'energy_transfer': energy_transfer_indirect_from_tof
}
del graph['two_theta']
del graph['scattered_beam']
del graph['Ltotal']
sc.show_graph(graph, simplified=True)

In [ ]:
def correct_tof(data):
    data = data.copy()
    # TODO according to Mads' script
    #  Instrument focuses at center of pulse, 2.86 ms / 2
    shift = sc.scalar(0.5*2.86, unit='ms')
    if 'tof' in data.coords:
        data.coords['tof'] -= shift.to(unit=data.coords['tof'].unit)
    if data.bins and 'tof' in data.bins.coords:
        data.bins.coords['tof'] -= shift.to(unit=data.bins.coords['tof'].bins.unit)
    return data

In [ ]:
def to_energy_transfer(data):
    data = correct_tof(data)
    return data.transform_coords('energy_transfer', graph=graph)

In [ ]:
unmasked = to_energy_transfer(events)
unmasked

In [ ]:
masked = to_energy_transfer(binned_for_mask)
masked

In [ ]:
masked_hist = masked.bins.concat().hist(energy_transfer=100)
unmasked_hist = unmasked.hist(energy_transfer=100)

In [ ]:
utils.add_variances(masked_hist)
utils.add_variances(unmasked_hist)

In [ ]:
pp.plot({
    'unmasked': unmasked_hist,
    'masked': masked_hist,
        })

# TODO
- save xye